## Práctica voluntaria

Cálculo de un índice TF-IDF para palabras de un texto

Tf-idf (Term frequency – Inverse document frequency)

En el fichero indiceTF-IDF.py he elaborado tres funciones que corresponden a los cálculos de estas tres fórmulas matemáticas guiandome por la información que obtuve en https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76. En el mismo fichero .py comento cada función con su significado.


$tf(t,d) = \frac{f(t,d)}{max\{f(t,d): t \epsilon d\}}$

$idf(t,D) = log \frac{|D|}{|\{d \epsilon D: t \epsilon d\}|} $

$tfidf(t,d,D)  = tf(t,d)· idf(t,D)$

Siendo: 
- $f(t,d)$ : la frecuencia bruta de t en un documento d de la colección D
- $|D|$ : número de elementos de la colección
- $|\{d \epsilon D: t \epsilon d \}|$ : número de documentos donde aparece t

También intenté elaborar una función genérica al completo para que se pudiera adaptar a cualquier input de textos, pero se me complicó un poco por las diferentes características que pudieran tener cada uno de estos textos. 

A continuación aplico las funciones al caso que se pide de los documentos .txt del campus.


In [1]:
# ejecutamos el fichero .py en nuestra misma carpeta
%run -i indiceTF-IDF.py

In [2]:
# nuestra lista de documentos de texto
files = [doc_reader("file01.txt"),doc_reader("file02.txt"),doc_reader("file03.txt")]
files

['casa casa perro gato nevera',
 'perro gato ordenador libro',
 'libro espada cuento ordenador']

Con este código vemos que es posible convertir nuestros elementos de 'files' en listas de palabras. Lo malo de esta estrategia es que se pierde toda información sobre la estructura del texto o frase.

In [3]:
N = len(files)
bagOfWords1 = files[0].split(' ')
bagOfWords2 = files[1].split(' ')
bagOfWords3 = files[2].split(' ')

En primer lugar escribo el proceso de obtener el índice Tf-idf sin hacer el correspondiente preprocessing con las palabras porque no estoy segura de la universalidad de las funciones que he definido para ello y por tanto pongo aquí ambos ejemplos.

Para analizar el indice Tf-idf de estas palabras, necesitaremos generar un diccionario que las contenga a todas, pero sin estar ninguna repetida. Así las juntamos todas las que salen al menos una vez en uno de los ficheros

In [4]:
uniqueWords = set(bagOfWords1).union(set(bagOfWords2)).union(set(bagOfWords3))
uniqueWords

{'casa', 'cuento', 'espada', 'gato', 'libro', 'nevera', 'ordenador', 'perro'}

Ahora necesitamos codificar cada uno de nuestros ficheros dentro de este diccionario de palabras con las que contiene cada uno y su frecuencia.

In [5]:
numOfWords1 = dict.fromkeys(uniqueWords,0)
for word in bagOfWords1:
    numOfWords1[word] += 1

numOfWords2 = dict.fromkeys(uniqueWords,0)
for word in bagOfWords2:
    numOfWords2[word] += 1

numOfWords3 = dict.fromkeys(uniqueWords,0)
for word in bagOfWords3:
    numOfWords3[word] += 1

In [6]:
# ejecutamos la primera función auxiliar como vimos en la introducción
tf1 = computeTF(numOfWords1, bagOfWords1)
tf2 = computeTF(numOfWords2, bagOfWords2)
tf3 = computeTF(numOfWords3, bagOfWords3)

In [7]:
# ejecutamos la segunda función auxiliar como vimos en la introducción
idfs= computeIDF([numOfWords1,numOfWords2,numOfWords3])

In [8]:
# ejecutamos la tercera función que es la que ya nos proporciona el índice final.
tfidf1 = computeTFIDF(tf1,idfs)
tfidf2 = computeTFIDF(tf2,idfs)
tfidf3 = computeTFIDF(tf3,idfs)

df = pd.DataFrame([tfidf1,tfidf2,tfidf3]) 
df

,nevera,libro,casa,ordenador,espada,perro,gato,cuento
0,0.219722,0.000000,0.439445,0.000000,0.000000,0.081093,0.081093,0.000000
1,0.000000,0.101366,0.000000,0.101366,0.000000,0.101366,0.101366,0.000000
2,0.000000,0.101366,0.000000,0.101366,0.274653,0.000000,0.000000,0.274653


Usando usando esta el *kit de herramientas de lenguaje natural de python* vemos que hay manera de identidicar las 'palabras inútiles' también llamadas **stopwords** de cada idioma.

El módulo de python que calcula este índice directamente y de manera mas eficiente que veremos al final seguramente evalua mejor el peso que se le debe dar a estas palabras, que no nos aportan información distintiva sobre un texto.

Tendría sentido hacer un preprocessing en el que tuvieramos funciones auxiliares como las definidas a continuación, teniendo en cuenta el tema de las **stopwords** y otros  como convertirlas todas a minúsculas, eliminar signos, palabras de una letra que no aportan información y convertir cada palabra a su raíz para poder estudiar la frecuencia de una manera mas amplia.


He tenido algunos problemas universalizando estas funciones para cualquier fichero.txt dado, pero siguen la idea que deberían. Para que sea mas cómodo incluyo estas funciones que son auxiliares también en mi fichero .py.

Para elaborar estas funciones auxiliares me he apoyado en la información hallada en: 
https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089


In [9]:
ges_bag1 = preprocess(bagOfWords1)
ges_bag2 = preprocess(bagOfWords2)
ges_bag3 = preprocess(bagOfWords3)

In [10]:
uniqueWords_prueba = set(ges_bag1).union(set(ges_bag2)).union(set(ges_bag3))
uniqueWords_prueba

{'cas', 'cuent', 'espad', 'gat', 'libr', 'never', 'orden', 'perr'}

In [11]:
numOfWords1_p = dict.fromkeys(uniqueWords_prueba,0)
for word in ges_bag1:
    numOfWords1_p[word] += 1

numOfWords2_p = dict.fromkeys(uniqueWords_prueba,0)
for word in ges_bag2:
    numOfWords2_p[word] += 1

numOfWords3_p = dict.fromkeys(uniqueWords_prueba,0)
for word in ges_bag3:
    numOfWords3_p[word] += 1

In [12]:
# ejecutamos la primera función auxiliar como vimos en la introducción
tf1_p = computeTF(numOfWords1_p, ges_bag1)
tf2_p = computeTF(numOfWords2_p, ges_bag2)
tf3_p = computeTF(numOfWords3_p, ges_bag3)

In [13]:
# ejecutamos la segunda función auxiliar como vimos en la introducción
idfs_p= computeIDF([numOfWords1_p,numOfWords2_p,numOfWords3_p])

In [14]:
# ejecutamos la tercera función que es la que ya nos proporciona el índice final.
tfidf1_p = computeTFIDF(tf1_p,idfs_p)
tfidf2_p = computeTFIDF(tf2_p,idfs_p)
tfidf3_p = computeTFIDF(tf3_p,idfs_p)

df_p = pd.DataFrame([tfidf1_p,tfidf2_p,tfidf3_p]) 
df_p

,cuent,libr,never,perr,gat,espad,orden,cas
0,0.000000,0.000000,0.219722,0.081093,0.081093,0.000000,0.000000,0.439445
1,0.000000,0.101366,0.000000,0.101366,0.101366,0.000000,0.101366,0.000000
2,0.274653,0.101366,0.000000,0.000000,0.000000,0.274653,0.101366,0.000000


Vemos que en este caso, por las características del ejemplo de textos, no hay diferencia entre los índices obtenidos anteriormente y los que hemos conseguido al realizar el preprocessing

La manera ya incorporada a python de calcular este índice, obtiene valores bastante distintos a mis funciones porque usa versión más _fina_ de este índice en la que además se tiene mucho más en cuenta dar poco peso a las _stopwords_ como dijimos antes.

In [15]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(files)
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df1= pd.DataFrame(denselist,columns = feature_names)
df1

,casa,cuento,espada,gato,libro,nevera,ordenador,perro
0,0.806032,0.000000,0.000000,0.306504,0.000000,0.403016,0.000000,0.306504
1,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.500000,0.500000
2,0.000000,0.562829,0.562829,0.000000,0.428046,0.000000,0.428046,0.000000
